In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

## Exploratory Data Analysis

In [ ]:
data.head(100)

In [ ]:
data.describe()

checking empty values

In [ ]:
data.isnull().sum()

In [ ]:
data.drop(['keyword','location'],axis=1)

In [ ]:
x_train=data.text
y_train=data.target


In [ ]:
test_data=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
pd.set_option('display.max_colwidth',None)    #setting the length of o/p to be printed
print(test_data['text'][0:20])

In [ ]:
x_test=test_data.text

In [ ]:
len(x_test)

In [ ]:
pd.set_option('display.max_colwidth',None)    #setting the length of o/p to be printed
x_test[50:80]

In [ ]:
print(x_train.head())
print(y_train.head())


##### TWEET ANALYSIS
* We can see different tweet contain @handle_name or mentioning to the people
* Many tweets contain some unrecognized character(typo)
* many of the tweets contain URL which are not of anys use for sentiment analysis.
* They all have the hashtag #SXSW or #sxsw.
* There is an html character reference for ampersand &amp.
* There were many non-english characters.
* There were many emoticons.
* There are some url links, some with http or https, and some without

### Pre-Processing done on tweets

In [ ]:
# LOwer case characters
x_train= x_train.str.lower()
x_test= x_test.str.lower()

### Removing URL links ###
I didn’t think URLs would help with sentiment analysis so I wanted to remove them. . I used two expressions, one for URLs with http or https, and a second for URLs without them, but with or without www.

In [ ]:
import re  #regex

In [ ]:
# train data
x_train = x_train.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
x_train.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)",'', x))  #Apply a function along an axis=0(by default) of the DataFrame

#test data
x_test = x_test.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
x_test.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)",'', x))



### Remove HTML reference characters ###
reference characters are not of any value to the analysis so they also should be removed.

In [ ]:
x_train = x_train.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
x_test = x_test.apply(lambda x: re.sub(r'&[a-z]+;', '', x))

### Remove Non-Letter characters ###
* Remove all the characters that weren’t letters, punctuation that is commonly used in emojis, or hash marks. There were a few non-english characters in the tweets.  They would add to the analysis so I wanted to remove them.
* Numbers typically don’t add much if any information so I wanted to remove them. Punctuation that isn’t usually associated with an emoji needed to be removed too because it didn’t add anything. I did all this with one regex.

In [ ]:
# x_train = x_train.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
# x_test = x_test.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

In [ ]:
x_train = x_train.apply(lambda x: re.sub(r"[^a-z\s]", '', x))
x_test = x_test.apply(lambda x: re.sub(r"[^a-z\s]", '', x))

In [ ]:

print(len(x_test))

max. possible length in all sentences

In [ ]:
type(x_train)

In [ ]:
def countSpace(string):
    lst=string.split()
    return len(lst)+10

In [ ]:
maxLen=x_train.map(countSpace).max()   # calculating max words possible in a string

In [ ]:
maxLen     # max length of a sentence in all strings

In [ ]:
ind=190
print(x_train[ind],y_train[ind])

In [ ]:
y_train.value_counts()     

Using  Pre-trained glove Word Embeddings vectors

In [ ]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='utf-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()    # strip remove trailing oe ending empty spaces in a string
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('../input/glove6b/glove.6B.100d.txt')

100 Dimensional embedding vectors are used

In [ ]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

importing some libraries and layers

In [ ]:
np.random.seed(0)
from keras.models import Model,load_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

all the strings have different length to pass in LSTM .The common solution to this is to use padding. Specifically, set a maximum sequence length, and pad all sequences to the same length.

### 2.3 - The Embedding layer

In Keras, the embedding matrix is represented as a "layer", and maps positive integers (indices corresponding to words) into dense vectors of fixed size (the embedding vectors). It can be trained or initialized with a pretrained embedding. In this part, you will learn how to create an [Embedding()](https://keras.io/layers/embeddings/) layer in Keras, initialize it with the GloVe 100-dimensional vectors loaded earlier in the notebook.

The `Embedding()` layer takes an integer matrix of size (batch size, max input length) as input. This corresponds to sentences converted into lists of indices (integers), as shown in the figure below.

<img src="images/embedding1.png" style="width:700px;height:250px;">
<caption><center> **Figure 4**: Embedding layer. This example shows the propagation of two examples through the embedding layer. Both have been zero-padded to a length of `max_len=5`. The final dimension of the representation is  `(2,max_len,100)` because the word embeddings we are using are 100 dimensional. </center></caption>

The largest integer (i.e. word index) in the input should be no larger than the vocabulary size. The layer outputs an array of shape (batch size, max input length, dimension of word vectors).

The first step is to convert all your training sentences into lists of indices, and then zero-pad all these lists so that their length is the length of the longest sentence. 

In [ ]:
import re # library to remove 

In [ ]:
def sentences_to_indices(X, word_to_index, max_len):
    
    m=X.shape[0] # number of training examples

    X_indices=np.zeros([m,max_len])

    for i in range(m):
        
        
#         sentence = re.sub('[\W_]+', '', X[i]) 
        sentence_words=[word.replace('\t','') for word in X[i].split(' ') if word.replace('\t','')!=''] # list of words in a sentence
#         sentence_words=[]
#         for word in X[i].split(' '):
#             if word.replace('\t','')!='':
#                 sentence_words.append(re.sub('[\W_]+', '',word.lower().replace('\t','')))  # to remove all the characters  except numbers and alphabets 
            
        j=0
        for word in sentence_words:
            if(word in word_to_index.keys()):
                X_indices[i,j]=word_to_index[word]
            else:
                X_indices[i,j]=400000  # last key
            
            j=j+1
            
    
    return X_indices   # matrix containing index of each word 
        
        
        
    

In [ ]:
len(x_test)

Let's build the Embedding() layer in Keras, using pre-trained word vectors. After this layer is built, you will pass the output of sentences_to_indices() to it as an input, and the Embedding() layer will return the word embeddings for a sentence.

##### Implement pretrained_embedding_layer(). You will need to carry out the following steps:

1. Initialize the embedding matrix as a numpy array of zeroes with the correct shape.
2. Fill in the embedding matrix with all the word embeddings extracted from word_to_vec_map.
3. Define Keras embedding layer. Use Embedding(). Be sure to make this layer non-trainable, by setting trainable = False when calling Embedding(). If you were to set trainable = True, then it will allow the optimization algorithm to modify the values of the word embeddings.
4. Set the embedding weights to be equal to the embedding matrix

In [ ]:
index_to_word[400000]

In [ ]:

avg=np.zeros([1,len(word_to_vec_map["cucumber"])])
for k,v in word_to_vec_map.items():
    avg=np.add(avg,v)
avg=avg/len(word_to_vec_map)

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 100)
    
    emb_matrix=np.zeros([vocab_len,emb_dim])
    
    for word,index in word_to_index.items():
        if word=='￥':
            emb_matrix[index,:]=avg    # average of word embeddings is taken for unknown word
        else:
            emb_matrix[index,:]=word_to_vec_map[word]
        
    embedding_layer=Embedding(vocab_len,emb_dim,trainable=False)
     # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

# implementing 2 layer lstm sequence classifier

The model takes as input an array of sentences of shape (m, max_len ) defined by input_shape

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def model1(input_shape,word_to_vec_map,word_to_index):
    
    """" Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
        ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape,dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map,word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128,return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.2)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128,return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.2)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(2,activation=None)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=[sentence_indices],outputs=X)
    
    ### END CODE HERE ###
    
    return model
    

In [ ]:
model=model1((maxLen,),word_to_vec_map,word_to_index)
model.summary()

In [ ]:
# initial_learning_rate = 0.01
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])

In [ ]:
x_train_indices = sentences_to_indices(x_train, word_to_index, maxLen)

In [ ]:
x_test_indices=sentences_to_indices(x_test, word_to_index, maxLen)

In [ ]:
x_test_indices.shape

In [ ]:
x_train_indices.shape

In [ ]:
x_train_indices

In [ ]:
y_train_oh=pd.get_dummies(y_train).values   #one_hot_encoding_of_y_train

In [ ]:
y_train_oh[:10]

In [ ]:
print(x_train_indices)
print(y_train_oh)

In [ ]:
model.fit(x_train_indices, y_train_oh, epochs = 80, batch_size = 32, shuffle=True)

In [ ]:
# Saving Model
model.save("model.h5")

In [ ]:
#Load the model
load_model("./model.h5")

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
print(len(x_train_indices))
print(len(x_test_indices))

In [ ]:
pred=model.predict(x_test_indices)
print(pred)
lst=[]
for i in range(len(pred)):
    if(pred[i][0]>pred[i][1]):
        lst.append(0)
    else:
        lst.append(1)

sample_submission["target"]=lst

INCORRECT PREDICTIONS

In [ ]:
cnt=0 
for i in range(len(lst)):
    if(y_train[i]!=sample_submission["target"][i]):
        cnt+=1
        print(x_train[i],sample_submission["target"][i])
        
    

In [ ]:
print(cnt)    

In [ ]:
sample_submission.head(10)

In [ ]:
sample_submission.to_csv("submission.csv", index=False)